<a href="https://colab.research.google.com/github/arthurrferroni/UNIFEI/blob/main/Copy_of_ECO904_2_4_Atividade_Avaliativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avalição de Técnicas via o uso de Métricas

Dado uma base de dados (*Dataset*) defina qual a melhor técnica de aprendizado de máquina que consiga reproduzir os padrões armazenados na base de dados.

Nesta atividade realize a 


## Base de Dados

Gerador de Base de dados do [Scikit-Learn (Sklearn)](https://scikit-learn.org/stable/) para um tutor com 3 classes [0, 1, 2].

**Atenção**: Use seu numero de matrícula no campo abaixo para gerar sua base de dados própria. Se esquecer de alterar esse campo o trabalho será anulado.

In [ ]:
from sklearn.datasets import make_classification
import pandas as pd

matricula = 2020017733 #@param {type:"number"}
num_atributos = 10

# gerando atributos:X e tutor:y
X, y = make_classification(
    n_samples=1000, 
    n_features=num_atributos, 
    n_classes=3, 
    n_informative=6,
    class_sep=0.5,
    random_state=matricula
    )

nome_atributos = [f'col{i+1}' for i in range(0,num_atributos)]

# criando DataFrame da Base de Dados
bd = pd.DataFrame(data=X, columns=nome_atributos)
bd['tutor'] = y
# exibindo informação da Base de Dados
print(bd.shape[0],'linhas x',bd.shape[1],'colunas')
bd.head()

1000 linhas x 11 colunas


,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,tutor
0,-1.490048,2.636417,-0.142315,0.399884,1.823919,-0.858275,1.036208,-0.884091,-0.437499,1.493653,1
1,0.477547,1.203082,-0.849951,0.195190,0.918414,0.077677,0.612476,-0.156711,-0.130598,-0.349950,1
2,-0.028831,0.731973,1.296082,1.654042,-0.877432,-1.287589,0.300887,0.445189,-1.012234,-0.044015,2
3,-0.170713,-1.418577,1.237180,-1.628896,0.199154,1.076015,-0.480571,-1.275830,1.818643,-0.316026,2
4,0.694830,1.375707,0.884580,1.186527,-0.450612,1.315482,1.152835,-1.029300,-0.983746,0.085258,1


## Classificadores com Parâmetros Base

Abaixo são analisadas as acurácias de acerto de [10 técnicas de aprendizado de máquina supervisionado](https://scikit-learn.org/stable/supervised_learning.html) e seus tempos de treinamento.

O [modelo de treinamento](https://scikit-learn.org/stable/modules/cross_validation.html) é dividido em conjunto de treinamento (train) e validação (test) para verificar a capacidade de generalização básica (acertar casos desconhecidos baseado no treinamento). 

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# classificadores para avaliação
classificadores = [
    KNeighborsClassifier(),
    LinearSVC(random_state=42),
    SVC(random_state=42),
    GaussianProcessClassifier(random_state=42),
    DecisionTreeClassifier(max_depth=5,random_state=42),
    RandomForestClassifier(max_depth=5,random_state=42),
    MLPClassifier(hidden_layer_sizes=(20,),max_iter=1000,random_state=42),
    AdaBoostClassifier(random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

# validação cruzada para seleção de modelo
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# medição de tempo
import time

# treinamento
res=[]
for clf in classificadores:
  inicio = time.time()
  clf.fit(X_train,y_train)
  fim = time.time()
  acuracia = clf.score(X_test,y_test)
  # salvando resultados
  res.append({
      'modelo':clf.__class__.__name__,
      'acurácia':acuracia,
      'tempo(s)':fim-inicio})

#impressão
pd.DataFrame(data=res)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


,modelo,acurácia,tempo(s)
0,KNeighborsClassifier,0.730,0.005576
1,LinearSVC,0.540,0.335365
2,SVC,0.710,0.175638
3,GaussianProcessClassifier,0.735,2.258996
4,DecisionTreeClassifier,0.580,0.014018
5,RandomForestClassifier,0.635,0.525272
6,MLPClassifier,0.730,4.668223
7,AdaBoostClassifier,0.535,0.258993
8,GaussianNB,0.545,0.010019
9,QuadraticDiscriminantAnalysis,0.595,0.016631


### Métricas de Avaliação

Os modelos matemáticos obtidos do treinamento são avaliados pelas métricas de [acurácia, precisão, recall, F1 e matriz de confusão](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

for clf in classificadores:
  print('_________________________________')
  print(f'{clf.__class__.__name__}\n')
  y_pred = clf.predict(X_test)
  print(classification_report(y_test,y_pred))
  print('Matriz de confusão:')
  print(confusion_matrix(y_test,y_pred),'\n\n')

_________________________________
KNeighborsClassifier

              precision    recall  f1-score   support

           0       0.64      0.83      0.72        59
           1       0.79      0.75      0.77        73
           2       0.79      0.62      0.69        68

    accuracy                           0.73       200
   macro avg       0.74      0.73      0.73       200
weighted avg       0.74      0.73      0.73       200

Matriz de confusão:
[[49  6  4]
 [11 55  7]
 [17  9 42]] 


_________________________________
LinearSVC

              precision    recall  f1-score   support

           0       0.48      0.49      0.49        59
           1       0.59      0.74      0.65        73
           2       0.52      0.37      0.43        68

    accuracy                           0.54       200
   macro avg       0.53      0.53      0.52       200
weighted avg       0.53      0.54      0.53       200

Matriz de confusão:
[[29 17 13]
 [ 9 54 10]
 [22 21 25]] 


_________________

# ATIVIDADES

## Perguntas

10% da nota da atividade por questão

1) Qual técnica do tópico "Classificadores com Parâmetros Base" teve maior acurácia?

In [ ]:
Resposta1 = "GaussianProcessClassifier" #@param {type:"string"}
print(Resposta1)

GaussianProcessClassifier


2) Considerando técnicas com acurácia maior que 70%, qual conseguiu um menor tempo de treinamento?

In [ ]:
Resposta2 = "KNeighborsClassifier" #@param {type:"string"}
print(Resposta2)

KNeighborsClassifier


3) No tópico "Métricas de Avaliação", qual técnica conseguiu a maior F1 para a classe 2?

In [ ]:
Resposta3 = "2 iguais" #@param {type:"string"}
print(Resposta3)

4) Qual classe, em todos os modelos treinados, obteve o maior acerto de 'Recall'?

In [ ]:
Resposta4 = "RandomForestClassifier" #@param {type:"string"}
print(Resposta4)

RandomForestClassifier


## Desenvolvimento

A partir do sorteio abaixo, verifique quais 2 técnicas de aprendizado de máquina supervisionado classificador serão escolhidas para a atividade:

In [ ]:
import random
# fixa semente com matrícula definida na 1a célula
random.seed(matricula)
# monta nomes das técnicas
tecnica_nomes = [c.__class__.__name__ for c in classificadores]
# embaralha técnicas
random.shuffle(tecnica_nomes)
# exibe técnicas sorteadas
for n in range(0,2):
  print('Técnica',n+1,'=',tecnica_nomes[n])

Técnica 1 = QuadraticDiscriminantAnalysis
Técnica 2 = GaussianProcessClassifier


### Técnica 1

*30% da nota da atividade*

Utilizando o [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) faça uma busca nos parâmetros da técnica para o melhor resultado da **menor** métrica *Recall* encontrada para cada classe.

Lista de métricas disponíveis para o parâmetro "scoring" do GridSearchCV:

In [ ]:
from sklearn.metrics import SCORERS
print('Métricas avaliadas:')
list(SCORERS.keys())

Métricas avaliadas:


['explained_variance',
 'r2',
 'max_error',
 'neg_median_absolute_error',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_root_mean_squared_error',
 'neg_mean_poisson_deviance',
 'neg_mean_gamma_deviance',
 'accuracy',
 'top_k_accuracy',
 'roc_auc',
 'roc_auc_ovr',
 'roc_auc_ovo',
 'roc_auc_ovr_weighted',
 'roc_auc_ovo_weighted',
 'balanced_accuracy',
 'average_precision',
 'neg_log_loss',
 'neg_brier_score',
 'adjusted_rand_score',
 'rand_score',
 'homogeneity_score',
 'completeness_score',
 'v_measure_score',
 'mutual_info_score',
 'adjusted_mutual_info_score',
 'normalized_mutual_info_score',
 'fowlkes_mallows_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'jaccard',
 'jaccard_macro',
 'jaccard_m

Definições de ajuste no GridSearchCV:
* a técnica escolhida é a 1a sorteada
* no mínimo 3 parâmetros devem ser ajustados.
* podem ser parâmetros numéricos ou discretos
* todos os parâmetros devem ser discretizados para a grade de resultados
* árvores devem ser restringidas com profundidade máxima de 6

In [ ]:
from sklearn.model_selection import GridSearchCV
# desativa avisos de execução
import warnings
warnings.filterwarnings("ignore")

# lista de parâmetros viáveis de ajuste buscado na técnica
parametros = {
    'hidden_layer_sizes':[(10,),(25,),(50,),(75,)],
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
}

# técnica a ser ajustada
tec1 = MLPClassifier(random_state=42)
# busca de parâmetros para um 5-Fold e métrica de pontuação 'Recall'
busca = GridSearchCV(tec1, parametros, cv=5, scoring='recall_micro')
# buscando no treinamento
busca.fit(X,y)
# 5 melhores resultados
pd.DataFrame(data=busca.cv_results_).sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
36,0.299318,0.085891,0.001963,0.001103,relu,"(10,)",lbfgs,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.785,0.780,0.700,0.725,0.790,0.756,0.036524,1
35,1.434025,0.476749,0.002287,0.000410,tanh,"(75,)",adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.710,0.790,0.710,0.785,0.770,0.753,0.035721,2
44,0.534749,0.014862,0.001207,0.000044,relu,"(50,)",adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.755,0.795,0.725,0.730,0.750,0.751,0.024779,3
47,0.551392,0.132784,0.001423,0.000411,relu,"(75,)",adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.770,0.805,0.725,0.730,0.675,0.741,0.043977,4
32,0.686662,0.008944,0.001458,0.000044,tanh,"(50,)",adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.715,0.795,0.715,0.750,0.725,0.740,0.030332,5


Relatório de classificação e matriz de confusão para o melhor ajuste:

In [ ]:
print('Melhores parâmetros:')
print('  ',busca.best_estimator_,'\n')
print('Relatório de Classificação:')
y_pred = busca.best_estimator_.predict(X)
print(classification_report(y,y_pred),'\n')
print('Matriz de Confusão:')
print(confusion_matrix(y,y_pred))

Melhores parâmetros:
   MLPClassifier(hidden_layer_sizes=(10,), random_state=42, solver='lbfgs') 

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.84      0.82      0.83       333
           1       0.83      0.84      0.83       336
           2       0.79      0.79      0.79       331

    accuracy                           0.82      1000
   macro avg       0.82      0.82      0.82      1000
weighted avg       0.82      0.82      0.82      1000
 

Matriz de Confusão:
[[274  23  36]
 [ 21 282  33]
 [ 33  35 263]]


### Técnica 2

*30% da nota da atividade*

Utilizando o [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) faça uma busca nos parâmetros da técnica para o melhor resultado da menor métrica F1 encontrada para cada classe.

Neste buscador é necessário definir uma distribuição nas variáveis inteiras seguindo as seguintes [funções de distribuição contíonuas](https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions).

Definições de ajuste no RandomizedSearchCV:
* a técnica escolhida é a 2a sorteada
* no mínimo 3 parâmetros devem ser ajustados.
* podem ser parâmetros numéricos ou discretos
* todos os parâmetros numéricos devem ser ligados a uma distribuição
* árvores devem ser restringidas com profundidade máxima de 6

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform,norm
# desativa avisos de execução
import warnings
warnings.filterwarnings("ignore")

# lista de parâmetros viáveis de ajuste buscado na técnica
parametros = {
    'n_estimators':[50,100,150,200],
    'criterion':['gini', 'entropy'],
    'max_features': ['auto', 'sqrt', 'log2'],
}

# técnica a ser ajustada
tec2 = RandomForestClassifier(max_depth=6,random_state=42)
# busca de parâmetros para um 5-Fold e métrica de pontuação 'Recall'
busca = RandomizedSearchCV(tec2, parametros, cv=5, scoring='f1_micro')
# buscando no treinamento
busca.fit(X,y)
# 5 melhores resultados
pd.DataFrame(data=busca.cv_results_).sort_values(by='rank_test_score').head()

Relatório de classificação e matriz de confusão para o melhor ajuste:

In [ ]:
print('Melhores parâmetros:')
print('  ',busca.best_estimator_,'\n')
print('Relatório de Classificação:')
y_pred = busca.best_estimator_.predict(X)
print(classification_report(y,y_pred),'\n')
print('Matriz de Confusão:')
print(confusion_matrix(y,y_pred))